# Adaptation summary

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig1D_habituation\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')
import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')
info=info[info.stimulusProtocol=='hab']
info

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 29      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['allowEpisodeSwitch']=1

info['expTime']=expTime


info

# physical interactions

In [ ]:
infoP=pd.read_csv(expFile, sep=',')
infoP=infoP[infoP.stimulusProtocol=='habTP']
infoP

In [ ]:
posPath = []
aviPath=[]
bdGroupAll=[]
bgAll=[]
expTimeAll=[]

for index,row in infoP.iterrows():
    #currTime=datetime.strptime(row.date, '%Y%m%d%H%M%S')
    DishDir=RawDataDir+row.path
    #DishDir='D:\\data\\b\\2017\\'+row.path
    
    #if not os.path.isdir(DishDir):
    #    DishDir='E:\\b\\2017\\'+row.path
    for root, dirnames, filenames in os.walk(DishDir):
        for filename in fnmatch.filter(filenames, '*nogaps*.txt'):
            posPath.append(os.path.join(root, filename))
            bdGroupAll.append('0 0')
            bgAll.append(row.bd)
            currTime=datetime.strptime(filename[-23:-4], '%Y-%m-%dT%H_%M_%S')
            expTimeAll.append(currTime)
        for filename in fnmatch.filter(filenames, '*split*.avi'):
            aviPath.append(os.path.join(root, filename))


infoPAll=pd.DataFrame({'txtPath': posPath})
infoPAll['epiDur']=29
infoPAll['bd']=bgAll
infoPAll['bdGroup']=bdGroupAll
infoPAll['episodes']=-1
infoPAll['inDish']=10
#info['stimulusProtocol']=8
infoPAll['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
infoPAll['minShift'] = 60 # minimum number of seconds to shift for control IAD
infoPAll['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
infoPAll['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
infoPAll['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
infoPAll['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
infoPAll['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
infoPAll['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
infoPAll['expTime']=expTimeAll

infoPAll['ProcessingDir']=ProcessingDir
infoPAll['outputDir']=outputDir
infoPAll['allowEpisodeSwitch']=1

infoPAll

In [ ]:
infoVirtualPhysical=pd.concat([info,infoPAll],sort=True)

csvFile=os.path.join(ProcessingDir,'Fig1D_habituation.csv')
infoVirtualPhysical.to_csv(csvFile,encoding='utf-8')
infoVirtualPhysical

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in infoVirtualPhysical.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])


df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp['animalIndexCont']=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1

df=df[df['animalIndex']!=14]
df=df[df['animalIndex'].isin(np.arange(0,14,2))]
df['episode']='skype'
print('df shape',df.shape)

d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]

t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

In [ ]:
df['c']=0
df.loc[df.animalSet>8,'c']=4
df.c[df.animalSet==1]=1
df.c[df.animalSet==2]=2
df.c[df.animalSet.isin([3,4,5,6,7,8])]=3

df['c2']=1
df.loc[df.animalSet>8,'c2']=0
df.c2[df.animalSet==1]=1
df.c2[df.animalSet==2]=1
df.c2[df.animalSet.isin([3,4,5,6,7,8])]=0



In [ ]:
sns.tsplot(data=df, time="t3",value="si",unit="animalIndexCont",condition='c',estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([8,22])
plt.ylim([0,.7])

In [ ]:
pfp.paper()
inToCm=2.54
plt.figure(figsize=(9/inToCm,4.5/inToCm))
ax = plt.gca()
sns.tsplot(data=df, time="t3",
           value="si",
           unit="animalIndexCont",
           condition='c',
           estimator=np.nanmean,
           interpolate=True,
           err_style="ci_bars",
           ax=ax)#,color=['k','r','r','r']);

ax.set_xlim([9,23])
ax.set_ylim([0,.8])
sns.despine()
ax.set_xlabel('time of day [hh]')
ax.set_ylabel('attraction index')
ax.set_title('adaptation')
plt.legend(ncol=2,loc='lower center')


In [ ]:
df['t4']=np.round(df.t3*2)/2.0
plt.figure(figsize=(9/inToCm,4.5/inToCm))
ax = plt.gca()
sns.tsplot(data=df.groupby(['animalIndexCont','c','t4']).mean().reset_index(), time="t4",
           value="si",
           unit="animalIndexCont",
           condition='c',
           estimator=np.nanmean,
           interpolate=True,
           err_style="ci_bars",
           ax=ax);

ax.set_xlim([9,23])
ax.set_ylim([0,.8])
sns.despine()
ax.set_xlabel('time of day [hh]')
ax.set_ylabel('attraction index')
ax.set_title('adaptation')
plt.legend(ncol=2,loc='lower center')

In [ ]:
ix=((df.c==3)&(~df.t4.isin(df[df.c==4].t4.unique()))).values
df2=df[~ix]

In [ ]:
plt.figure(figsize=(7/inToCm,4.5/inToCm))
ax = plt.gca()
sns.tsplot(data=df.groupby(['t4','c2','animalIndexCont']).mean().reset_index(), time="t4",
           value="si",
           unit="animalIndexCont",
           condition='c2',
           estimator=np.nanmean,
           interpolate=True,
           #err_style="ci_bars",
           #ci='sd',
           ax=ax,
          color=['gray','k']);

ax.set_xlim([9,23])
ax.set_ylim([0,1])
sns.despine()
ax.set_xlabel('Time of day')
ax.set_ylabel('Attraction')
ax.set_yticks([0,0.5,1])

ax.set_xticks([10,14,18,22])
ax.set_xticklabels(['10am','2pm','6pm','10pm'])

#ax.set_title('adaptation')
L=plt.legend(loc='upper right',title='Interaction')
L.get_texts()[0].set_text('Physical')
L.get_texts()[1].set_text('Virtual')


In [ ]:
plt.figure(figsize=(7/inToCm,4.5/inToCm))
ax = plt.gca()
sns.tsplot(data=df2.groupby(['t4','c2','animalIndexCont']).mean().reset_index(), time="t4",
           value="si",
           unit="animalIndexCont",
           condition='c2',
           estimator=np.nanmean,
           interpolate=True,
           #err_style="ci_bars",
           #ci='sd',
           ax=ax,
          color=['gray','k']);

ax.set_xlim([9,23])
ax.set_ylim([0,1])
sns.despine()
ax.set_xlabel('Time of day')
ax.set_ylabel('Attraction')
ax.set_yticks([0,0.5,1])

ax.set_xticks([10,14,18,22])
ax.set_xticklabels(['10am','2pm','6pm','10pm'])

#ax.set_title('adaptation')
L=plt.legend(loc='upper right',title='Interaction')
L.get_texts()[0].set_text('Physical')
L.get_texts()[1].set_text('Virtual')
figPath=outputDir+'\\1D_physicalVsVirtual_adaptation.svg'
plt.savefig(figPath)

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts



for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)


from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts


for i,row in infoPAll.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\bgMed*.csv')[0])
    copyList.append(glob.glob(head+'\\*nogaps.txt')[0])
    #copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    aviTime=glob.glob(head+'\\*.avi')[0][-18:-4]
    aviTime=datetime.strptime(aviTime,'%Y%m%d%H%M%S')


    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
        #os.chdir(toDirectory)
        if 'nogaps.txt' in f:
            
            old=glob.glob(toDirectory+'\\*nogaps.txt')[0]
            #t=datetime.strftime(row.expTime, '%Y-%m-%dT%H_%M_%S')
            t=datetime.strftime(aviTime, '%Y-%m-%dT%H_%M_%S')
            #t=aviTime#, '%Y%m%d%H%M%S'
            new=old[:-4]+str(i).zfill(2)+"_"+t+'.txt'
            os.rename(old,new)
            print(new)